# Pipulate: Learn Python through SEO Investigations...

## Global Modules & Google Login (*Don't change*)

In [ ]:
from imp import reload              # This keeps your edits to functions.ipynb in-memory
import logging; reload(logging); logger = logging.getLogger()
import re, sys, os, shelve, logging # Standard Library imports (where I'm not using "from")
from collections import namedtuple  # For creation of informative APIs for workflows, etc.
import requests                     # Third-Party Library imports
import notebook_finder              # This allows .ipynb files to be loaded like as modules
import goodsheet; reload(goodsheet); import goodsheet # OAuth & Google Services / reload intentional
#import warnings; warnings.filterwarnings('ignore')

## Interfaces For Use In Workflows (*Don't change*)

In [ ]:
Configuration = namedtuple('Configuration', 'oauth_file, repeat_seconds, rows_to_batch, chunk_limit')
Pipulate = namedtuple('Pipulate', 'sheet tab args_tab data cache') # Per-row lookups on table
Populate = namedtuple('Populate', 'sheet tab list_of_rows cols')   # Initialize a new table
Append   = namedtuple('Append', 'sheet tab list_of_rows')          # Append new rows

## Configuration & Workflows (*Study, then change*)
- This is where we name a configuration and a workflow. Their names should usually match.
- Standard system behavior is acheivecd by setting both configuration and workflow to "default".
- Configurations is a dict of environment variables that can be picked from by setting the configuration value.
- Workflows are lists of ***named tuples*** named either Pipulate(), Populate() or Append().
- Pipulate() pipulates pre-existing rows, while Populate() populates rows in the first place.
- The example below actually has ***multiple workflows*** to choose from my use of a dicitonary key.
- After studying these examples, you can design your own workflows. These two are very simple common needs.
- String Populate() and Pipualte() together in the same workflow to carry out complex multi-step investigations.

## Job Configuration

In [ ]:
# First, we set up the variable values that are going to be used in the workflows.

workflow_name = "default"
configuration = "default"

# These are abbreviations that can be used in the workflows below.
sheet_name = "Testing"
tab_name = "Sheet1"
dates = goodsheet.common_date_boundaries()
start = dates.start_30_days
end   = dates.end_30_days
import private       # <-- Put info you want to keep out of the git repo in here.
site = private.site  # <-- Put the full URL of your site for Google Analytics here.
feed = private.feed  # <-- private.py can be used to keep arbitrary values private from repo.
gaid = private.gaid  # <-- Put your Google Analytics ProileID here (leave off "ga:")

configurations = {
    "default": Configuration(
        oauth_file='oauth.dat',
        repeat_seconds=None,
        rows_to_batch=50,
        chunk_limit=20
    ),
    "monitor": Configuration(
        oauth_file='oauth.dat',
        repeat_seconds=3600,
        rows_to_batch=50,
        chunk_limit=20
    )
}

goodsheet.filename = configurations[configuration].oauth_file
repeat_seconds = configurations[configuration].repeat_seconds
rows_to_batch = configurations[configuration].rows_to_batch
chunk_limit = configurations[configuration].chunk_limit

workflows = {
    "default":[  
        Pipulate(
            sheet=sheet_name,
            tab=tab_name,
            args_tab=None,
            data=None,
            cache=False
        )
    ],
    "seoinit":[
        Populate(
            sheet=sheet_name,
            tab="GSC",
            list_of_rows = "functions.populate_from_gsc('"+site+"', '"+start+"', '"+end+"')",
            cols = None
        ),
        Populate(
            sheet=sheet_name,
            tab="GA",
            list_of_rows = "functions.populate_from_ga('"+site+"','ga:"+gaid+"', '"+start+"', '"+end+"')",
            cols = None
        ),
        Populate(
            sheet=sheet_name,
            tab="Crawl",
            list_of_rows = "functions.one_page_crawl('"+site+"')",
            cols = None
        )
    ],
    "crawl":[
        Populate(
            sheet=sheet_name,
            tab="Crawl",
            list_of_rows = "functions.one_page_crawl("+site+"')",
            cols = None
        )
    ],
    "monitor":[
        Append(
            sheet="APMonitor",
            tab="Feed",
            list_of_rows = "functions.populate_from_atom('APMonitor', 'Feed', '"+feed+"')",
        )
    ],
}
workflow = workflows[workflow_name]

## Debugging Tools

In [ ]:
def error(exception_info, stop=True, info=None):
    """Prints informative error and stops execution unless optional stop argument is False."""
    exception_type, description, traceback = exception_info
    errorname = exception_type.__name__
    filename = os.path.split(traceback.tb_frame.f_code.co_filename)[1]
    linenumber = traceback.tb_lineno
    if info:
        description = info
    #print("ERROR: %s: %s in %s line %s" % (errorname, description, filename, linenumber))
    logger.exception("ERROR:")
    if stop:
        raise SystemExit()

## The Pipulate() Process

In [ ]:
def pipulate(google_sheet_name, google_sheet_tab, tab_with_args=None, data=None, cache=True):
    """Update all cells in Google Sheet where columns are named with pipulate functions.
    
    The pipulate function is defines the conventional behavior of using GSheets for I/O.
    It scans from left-to-right and top-to-bottom of a Google Sheet, finding "requests".
    Requests are empty cells in columns labeled with function names found in functions module.
    Pipulate will execute the named function using values from that row as function arguments.
    The entire row's contents are fed into the named function as key/value pairs as **kwargs.
    If **kwargs proves to be too messy, decorators like @url can clean up the passed-in args.
    Results are batched-up and the sheet updated on every "rows_to_batch" number of rows.
    Because pipulate works against pre-existing rows, we must often create some rows first.
    More complex workflows can chain-up a series of row-creations and subsequent pipulations."""
    
    if not google_sheet_tab and not google_sheet_name:
        print("A Google Sheet and Tab name must be set. Please set and re-run.")
        raise SystemExit()
    
    # Create connection to GSheets, gather important details, and fail immediately on trouble.
    print('Examining "%s" in "%s"...' % (google_sheet_tab, google_sheet_name))
    try:
        worksheet = goodsheet.oauth().open(google_sheet_name).worksheet(google_sheet_tab)
        rows = worksheet.row_count
        cols = worksheet.col_count
        end_range = worksheet.get_addr_int(rows, cols)
    except:
        error(sys.exc_info())
    
    # Make a grab for additional key/value pair arguments (like API keys) from optional tab.
    arg_dict = {}
    arg_cells = None
    if tab_with_args:
        print('Looking for optional key/value pairs in "%s" tab...' % (tab_with_args))
        try:
            arg_sheet = goodsheet.oauth().open(google_sheet_name).worksheet(tab_with_args)     
            arg_cells = arg_sheet.range("A1:B20")
        except:
            error(sys.exc_info())
    if arg_cells:
        arg_temp = None
        for an_arg in arg_cells:
            if an_arg.value:
                if an_arg.col == 1:
                    arg_temp = an_arg.value
                else:
                    if an_arg.value:
                        arg_dict[arg_temp] = an_arg.value
                    else:
                        arg_dict[arg_temp] = None
                        arg_temp = None
            else:
                break

    # Create dictionary of pipulate functions invokable using their string-names dict keys!
    pipulate_funcs = [x for x in dir(functions) if x[0] is not '_']
    func_dict = {x.lower():eval('functions.%s' % x) for x in pipulate_funcs}

    # Create list of all the values found in the 1st row so we can identify named functions.
    row1_range = 'A1:%s' % worksheet.get_addr_int(1, cols)
    cell_range = worksheet.range(row1_range)
    col_names = [x.value.lower() for x in cell_range]

    # Find the first blank row in the sheet so we can always efficiently start in the right place.
    list_of_rows = worksheet.get_all_values() #Expensive but worth it 
    first_row_with_blank = rows #Default to end-of-sheet, unless actual blanks are found.
    if not list_of_rows:
        print("Error: Sheet empty. Please set initial values.")
        raise SystemExit()
    if len(list_of_rows) == 1:
        first_row_with_blank = 2 #If we only find 1 row (at all), then row 2 MUST be 1st blank.
    for row_dex, arow in enumerate(list_of_rows):
        for cell_dex, acell in enumerate(arow):
            if not acell and cell_range[cell_dex].value in func_dict: #Blank must be in func col.
                first_row_with_blank = row_dex+1
                break
        else:
            continue
        break
    list_of_rows = None #Blank no longer necessary potentially large object (be kind to memory)
    
    # Chunk the sheet into a series of ranges that will be used for batch-updates
    chunk_ranges = [(x+1, x+rows_to_batch+1) for x in list(range(rows-1)) if x%rows_to_batch == 0]
    chunks_to_go = [(x,y) for x,y in chunk_ranges if y > first_row_with_blank]
    
    # Pipulate each chunk
    for chunk_dex, (row_start, row_end) in enumerate(chunks_to_go):
        if chunk_limit > 0 and chunk_dex >= chunk_limit:
            break
        
        # Convert each chunk into Excel-like A2:B2 range notation
        top_left = worksheet.get_addr_int(row_start+1, 1)
        lower_right = worksheet.get_addr_int(row_end, cols)
        range_string = "%s:%s" % (top_left, lower_right)
        
        # Calculate correct number of rows in the last (likely) uneven chunk
        if chunk_dex+1 == len(chunk_ranges):
            range_string = "%s:%s" % (top_left, end_range)
        print("Pipulating range %s of %s (%s)" % (chunk_dex+1, len(chunks_to_go), range_string))
        print("%s updating in: " % range_string, end="")

        # Use chunk range to create new GSpread worksheet obj for writing-back response values.
        try:
            chunk_range = worksheet.range(range_string)
        except:
            error(sys.exc_info())
            
        # Step through each cell in the current chunk.
        row_dict = {} #Will contain all values from row and be passed to function as **kwargs
        count_down = int(len(chunk_range)/cols)+1
        for cell_dex, acell in enumerate(chunk_range): #Working with cells, but interested in rows
            row, col, val = acell.row, acell.col, acell.value
            # Now we pipulate this row's row_dict with column-name/cell-value pairs
            row_dict[col_names[col-1]] = val
            if col%cols == 0: #This is how we determine reaching last cell of row
                count_down = count_down - 1
                print("%s, " % count_down, end="")
                requests_response = None #HTML-cache object loop-leak prevention
                Response = None #Function-return loop-leak prevention
                # Why fetch the HTML for a URL more than once, if you don't have to?
                if 'url' in row_dict:
                    with shelve.open('urls') as urls:
                        if cache == True and row_dict['url'] in urls.keys():
                            requests_response = urls[row_dict['url']]
                        else:
                            try:
                                requests_response = requests.get(row_dict['url'])
                                urls[row_dict['url']] = requests_response #The moment of pickling
                            except requests.exceptions.RequestException as e:
                                print("(bad url) ", end="")
                                continue
                        # We now make the ENTIRE response object available to pipulate functions.
                        row_dict['response'] = requests_response
                # If data exists, we make it mutably available (for memory) to every row
                if data:
                    row_dict['data'] = data
                for key, val in row_dict.items():
                    if not val: #Only process empty cells in function-named columns
                        if key in [x.lower() for x in dir(functions) if x[0] is not '_']:
                            if arg_dict:
                                row_dict = {**arg_dict, **row_dict}
                                if '' in row_dict:
                                    del row_dict['']
                            try:
                                Response = func_dict[key](**row_dict) #pipulate!
                            except:
                                error(sys.exc_info(), info=key)
                            # We pipulate at end-of-row but update back to earlier cells from row.
                            row_start = cell_dex-cols+1
                            func_dex = col_names.index(key)
                            update_cell = row_start + func_dex
                            if Response:
                                if Response.ok:
                                    chunk_range[update_cell].value = Response.text #uncommitted
                                else:
                                    chunk_range[update_cell].value = "Err: %s" % Response.status_code
                            else:
                                print("Pipulate funcs must return Response object. Check %s function." % key)
                                error(sys.exc_info(), info=key)
                            row_dict = {} # Blank the now-used row_dict just for good measure
        try:
            # Batch update Google Sheets with the modified chunk_range.
            worksheet.update_cells(chunk_range)
            print("Range updated!")
        except:
            print(bad_api_message)
    print("Pipulation complete!") #do a little dance
    worksheet = None

## The Populate() Process

In [ ]:
def populate(sheet, tab, therows, cols=None):
    """Creates a new Tab in Google Sheet. Still must add chunking support."""
    print('Checking for "%s" in "%s"...' % (tab, sheet))
    if type(therows) is str:
        therows = eval(therows)
    elif type(therows) is not list:
        print("list_of_rows must be a list of lists or string that will eval to one.")
        raise SystemExit()
    try:
        spread = goodsheet.oauth().open(sheet)
        list_of_sheets = spread.worksheets()
    except:
        error(sys.exc_info())
    sheets = [x.title for x in list_of_sheets]    
    list_of_lists = therows
    if cols and len(cols) == len(rows[0]):
        headers = [cols]
        list_of_lists = headers + rows
    rows = len(list_of_lists)
    cols = len(list_of_lists[0])
    try:
        if tab not in sheets:
            print("Creating %s tab..." % tab)
            spread.add_worksheet(tab, rows, cols)
            worksheet = goodsheet.oauth().open(sheet).worksheet(tab)
        else:
            print("Resetting %s tab..." % tab)
            worksheet = goodsheet.oauth().open(sheet).worksheet(tab)
            worksheet.resize(rows=2, cols=cols)
            end_row_two = worksheet.get_addr_int(2,cols)
            blank_range = "A2:%s" % end_row_two
            cells_to_blank = worksheet.range(blank_range)
            for a_cell in cells_to_blank:
                a_cell.value = ''
            worksheet.update_cells(cells_to_blank)
            worksheet.resize(rows=rows, cols=cols)
        end_range = worksheet.get_addr_int(rows, cols)
    except:
        error(sys.exc_info())
    range_string = "A1:%s" % end_range
    chunk_range = worksheet.range(range_string)
    flat_list = [item for sublist in list_of_lists for item in sublist]
    chunk_ranges = [(x+1, x+rows_to_batch+1) for x in list(range(rows-1)) if x%rows_to_batch == 0]
    for chunk_dex, (row_start, row_end) in enumerate(chunk_ranges):
        if chunk_limit > 0 and chunk_dex >= chunk_limit:
            break
    
        # Convert each chunk into Excel-like A2:B2 range notation
        top_left = worksheet.get_addr_int(row_start, 1)
        lower_right = worksheet.get_addr_int(row_end, cols)
        range_string = "%s:%s" % (top_left, lower_right)
        #print((top_left, lower_right, range_string))

        # Calculate correct number of rows in the last (likely) uneven chunk
        if chunk_dex+1 == len(chunk_ranges):
            range_string = "%s:%s" % (top_left, end_range)

        # Use chunk range to create new GSpread worksheet obj for writing-back response values.
        try:
            chunk_range = worksheet.range(range_string)
        except:
            error(sys.exc_info())

        # Step through each cell in the current chunk.
        for cell_dex, acell in enumerate(chunk_range): #Working with cells, but interested in rows
            row, col, val = acell.row, acell.col, acell.value
            flat_cell_to_update = row*cols-(cols-col)-1
            #print((flat_cell_to_update, flat_list[flat_cell_to_update]))
            acell.value =  flat_list[flat_cell_to_update]
        try:
            # Batch update Google Sheets with the modified chunk_range.
            worksheet.update_cells(chunk_range)
            print("Range %s of %s updated!" % (chunk_dex+1, len(chunk_ranges)))
        except:
            print(bad_api_message)
            
    print("%s in %s populated." % (tab, sheet))

## The Append() Process

In [ ]:
def append(sheet, tab, therows):
    print('Checking for "%s" in "%s"...' % (tab, sheet))
    if type(therows) is str:
        therows = eval(therows)
    elif type(therows) is not list:
        print("list_of_rows must be a list of lists or string that will eval to one.")
        raise SystemExit()
    if not therows:
        print("No rows to append.")
        return
    try:
        spread = goodsheet.oauth().open(sheet)
        list_of_sheets = spread.worksheets()
    except:
        error(sys.exc_info())
    sheets = [x.title for x in list_of_sheets]    
    if tab in sheets:
        print('Appending to "%s" in "%s"...' % (tab, sheet))
        try:
            worksheet = goodsheet.oauth().open(sheet).worksheet(tab)
            rows = worksheet.row_count
            cols = worksheet.col_count
            append_width = len(therows[0])
            append_rows = len(therows)
            begin_range = worksheet.get_addr_int(rows+1, 1)
            end_range = worksheet.get_addr_int(rows+append_rows, cols)
        except:
            error(sys.exc_info())
        if append_width != cols:
            print("The %s tab must be the same width as the list_of_lists you're trying to append." % tab)
            raise SystemExit()
        new_size = rows + append_rows
        try:
            worksheet.resize(rows=new_size, cols=cols)
        except:
            error(sys.exc_info())
        range_string = "%s:%s" % (begin_range, end_range)
        cell_list = worksheet.range(range_string)
        flat_list = [item for sublist in therows for item in sublist]
        for cell_dex, acell in enumerate(cell_list): #Working with cells, but interested in rows
            acell.value =  flat_list[cell_dex]
        try:
            worksheet.update_cells(cell_list)
            print("List appended!")
        except:
            print(bad_api_message)
    else:
        print("%s tab not found. Appending only works when there is a pre-existing table." % tab)


## Schedulable Loop

In [ ]:
def schedule_me(workflow_name, workflow, repeat_seconds=False):
    goodsheet.oauth()
    print("WORKFLOW BEGINNING: %s" % workflow_name)
    for a_tuple in workflow:
        tuple_name = type(a_tuple).__name__
        if tuple_name == 'Pipulate':
            print("Pipulate request being processed...")
            sheet, tab, args_tab, data, cache = a_tuple
            pipulate(sheet, tab, args_tab, data, cache)
        elif tuple_name == 'Populate':
            print("Populate request being processed...")
            sheet, tab, list_of_rows, cols = a_tuple
            populate(sheet, tab, list_of_rows, cols)
        elif tuple_name == 'Append':
            print("Append request being processed...")
            sheet, tab, list_of_rows = a_tuple
            append(sheet, tab, list_of_rows)
    print("WORKFLOW COMPLETE!")
    if repeat_seconds:
        goodsheet.oauth()
        print("Setting a %s second loop for next workflow." % repeat_seconds)
        import sched, time
        scheduler = sched.scheduler(timefunc=time.time, delayfunc=time.sleep)
        scheduler.enter(delay=repeat_seconds, priority=1, action=schedule_me, argument=(workflow_name, workflow, repeat_seconds))
        scheduler.run()


## Main

In [ ]:
if __name__ == '__main__':
    logging.basicConfig(filename='debug.log', level=logging.CRITICAL)
    import functions; reload(functions); import functions #2nd import intentional for Notebook
    print("You are logged in as %s" % goodsheet.get_email())
    if 'workflow' in dir(functions):
        workflow = functions.workflow
    schedule_me(workflow_name, workflow, repeat_seconds)
    # All errors get this message, so check filename & tabname carefully & read tracebacks
    bad_api_message = "Can't reach Spreadsheet or nothing to pipulate. Double-check & try again."
        